In [17]:
!pip install google-colab-selenium

## **Importando as libs**

In [18]:
import google_colab_selenium as gs
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re

#### Acessar o google Drive

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Funções Auxiliares**

#### Converter altura

In [20]:
def feet_to_m(df, altura_feet, nova_coluna):
    # Limpa e separa pés e polegadas
    altura_limpa = df[altura_feet].str.replace('"', '').str.replace(' ', '')
    partes = altura_limpa.str.split("'", expand=True)

    # Converte para números
    pes = partes[0].astype(float)
    polegadas = partes[1].astype(float)

    # Calcula altura em metros
    df[nova_coluna] = (pes * 0.3048 + polegadas * 0.0254).round(2)

    return df

#### Converte peso

In [21]:
def lbs_to_kg(df, coluna_lbs, nova_coluna):
    df[nova_coluna] = df[coluna_lbs].str.replace(r'\D', '', regex=True).astype(float) * 0.45359237

    # Arredonda para 2 casas decimais
    df[nova_coluna] = df[nova_coluna].round(1)

    return df

## **Importação e Tratamento dos Dados**

#### Gerando DataFrame vazio com as colunas desejadas

In [22]:
# Colunas desejadas
columns = ["anoTorneio",
           "pais",
           "nome",
           "posicao",
           "time",
           "numeroCamiseta",
           "batThrow",
           "altura",
           "peso",
           "dataNascimento",
           "localNascimento"]

df_main = pd.DataFrame(columns=columns)

print(df_main)

Empty DataFrame
Columns: [anoTorneio, pais, nome, posicao, time, numeroCamiseta, batThrow, altura, peso, dataNascimento, localNascimento]
Index: []


#### Processo de importação das informações dos elencos - (2022 e 2025)

In [23]:
def extract_over_2022(url, ano_torneio):
  # Inicia contato com o drive
  driver = gs.Chrome()

  # Lista para armazenar os dados
  data_roster = []

  # Acessa a página
  driver.get(url)

  time.sleep(5) # Carrega a página

  # Localiza o elenco principal
  roster_view = driver.find_element(By.ID, "roster-view")

  # Localiza os containers
  sections = roster_view.find_elements(By.CLASS_NAME, "RosterSection__Container-sc-jpp1mx-1")

  # print(f"Total de grupos de jogadores: {len(sections)}")

  # Itera sobre cada container
  for section in sections:
    # Encontrando o título (posição) dentro do container maior
    position = section.find_element(By.XPATH, "./preceding-sibling::h4").text

    # Caso seja jogador
    if position != 'Managers & Coaches':

      # Acha cada jogador de cada container
      players = section.find_elements(By.CLASS_NAME, "RosterPersonView__BioInfo-sc-1w3lglk-1")

      # print(f"Total de jogadores em cada grupo: {len(players)}")

      # Itera sobre cada jogador
      for p in players:

        # Extrai nome do jogador
        name = p.find_element(By.CLASS_NAME, "RosterPersonView__PersonName-sc-1w3lglk-2").text

        # Faz print do nome e posição
        # print(f"Nome: {name}")
        # print(f"Posicao: {position}")
        player_data = {"nome": name, "posicao": position} # ADICIONA INFO PRO FUTURO DF

        # Extrai a quantidade do componente de linha (Row)
        rows = p.find_elements(By.CLASS_NAME, "InfoCompnents__InfoRow-sc-wyxo6k-0")

        # Iterando sobre cada linha
        for row in rows:

          # Extrai a quantidade do componente do container dentro da linha
          infos = row.find_elements(By.CLASS_NAME, "InfoCompnents__InfoContainer-sc-wyxo6k-1")

          # Iterando sobre cada container
          for info in infos:

            # Extrai o label (tipo da informação, como "Date of Birth")
            label = info.find_element(By.CLASS_NAME, "InfoCompnents__InfoLabel-sc-wyxo6k-2").text
            # Extrai os dados (como a data de nascimento, local de nascimento, etc.)
            data_info = info.find_element(By.CLASS_NAME, "InfoCompnents__InfoData-sc-wyxo6k-3").text

            player_data[label] = data_info # ADICIONA INFO PRO FUTURO DF

            # Faz print da label e o seu dado
            # print(f"{label}: {data_info}")

        # print("-" * 40)  # Separador para melhor visualização
        data_roster.append(player_data)  # ADICIONA INFO PRO FUTURO DF

    #Caso seja treinador
    else:

      # Acha cada jogador de cada container
      coaches = section.find_elements(By.CLASS_NAME, "RosterCoachView__Container-sc-1ll7e6h-0")
      # print(f"Total de técnicos: {len(coaches)}")

      for c in coaches:

        # Extrai nome do jogador
        name = c.find_element(By.CLASS_NAME, "RosterCoachView__PersonName-sc-1ll7e6h-3").text
        position = c.find_element(By.CLASS_NAME, "RosterCoachView__StatusInfo-sc-1ll7e6h-2").text

        # Faz print do nome e posição
        # print(f"Nome: {name}")
        # print(f"Posicao: {position}")
        # print("-" * 40)  # Separador para melhor visualização

        coach_data = {"nome": name, "posicao": position}
        data_roster.append(coach_data)  # ADICIONA INFO PRO FUTURO DF

  # Criando o DataFrame a partir dos dados coletados
  df = pd.DataFrame(data_roster)

  # Usando expressões regulares para extrair altura (em pés e polegadas) e peso
  df['altura'] = df['H/W'].str.extract(r'(\d+\' \d+")')
  df['peso'] = df['H/W'].str.extract(r'(\d+)$')
  df['numeroCamiseta'] = df['nome'].str.extract(r'(\d+)$') # Extraindo o número do jogador
  df['nome'] = df['nome'].str.replace(r' \d+$', '', regex=True).str.strip()
  df["anoTorneio"] = ano_torneio # Adicionando o ano do torneio

  # Renomeando várias colunas
  df = df.rename(columns={
      'B/T': 'batThrow',
  })

  df = df.drop(['W-L', 'ERA', 'AVG', 'HR', 'RBI', 'H/W'], axis=1)

  # Exibe as primeiras linhas para verificação
  # print(df.head())



  # Retorna concatenado
  return pd.concat([df_main, df], ignore_index=True)

  # Fechar o driver
  driver.quit()

#### Processo de importação das informações dos elencos - (2016)

In [24]:
def extract_equal_2016(url, ano_torneio):
  # Inicia contato com o drive
  driver = gs.Chrome()

  # Lista para armazenar os dados
  data_roster = []

  # Acessa a página
  driver.get(url)

  time.sleep(5) # Carrega a página

  # Localiza a tabela pelo ID "roster"
  table = driver.find_element(By.ID, "roster")

  # Extrai linhas da tabela
  rows = table.find_elements(By.TAG_NAME, "tr")

  # Lê cabeçalhos
  # headers = [th.text.strip() for th in rows[0].find_elements(By.TAG_NAME, "th")] # Vou setar na mão isso

  # Lê dados
  for row in rows[1:]:  # pula o cabeçalho
      # Verifica se é uma linha de posição (ex: Catchers)
      if "position" in row.get_attribute("class"):
          pos_cell = row.find_element(By.TAG_NAME, "td")
          current_position = pos_cell.text.strip()
      else:
          cols = row.find_elements(By.TAG_NAME, "td")
          if len(cols) == 6:
              player_data = [col.text.strip() for col in cols]
              player_data.insert(0, current_position)  # Adiciona a posição no início
              data_roster.append(player_data)

  # Define as colunas (incluindo "Position")
  columns = ["posicao", "nome", "numeroCamiseta", "batThrow", "altura", "peso", "dataNascimento"]

  # Cria o DataFrame
  df = pd.DataFrame(data_roster, columns=columns)

  df["anoTorneio"] = ano_torneio # Adicionando o ano do torneio

  # Exibe as primeiras linhas para verificação
  # print(df.head())

  # Retorna concatenado
  return pd.concat([df_main, df], ignore_index=True)

  # Fechar o driver
  driver.quit()

#### Processo de importação das informações dos elencos - (2012 e 2013)

In [25]:
def extract_before_2013(url, ano_torneio):

  # Inicia contato com o drive
  driver = gs.Chrome()

  # Lista para armazenar os dados
  data_roster = []

  # Acessa a página
  driver.get(url)

  time.sleep(5) # Carrega a página

  # Localiza a tabela pelo ID "roster"
  table = driver.find_element(By.ID, "roster_grid")


  # Extrai linhas da tabela
  rows = table.find_elements(By.TAG_NAME, "tr")

  # Lê cabeçalhos
  headers = [th.text.strip() for th in rows[0].find_elements(By.TAG_NAME, "th")]

  # Lê dados
  data = []
  for row in rows[1:]:  # pula o cabeçalho
      cols = [td.text.strip() for td in row.find_elements(By.TAG_NAME, "td")]
      if cols:
          data.append(cols)

  # Converte para DataFrame
  df = pd.DataFrame(data, columns=headers)

  df["anoTorneio"] = ano_torneio # Adicionando o ano do torneio

  # Inverte o nome do jogador
  df['Name'] = df['Name'].apply(lambda x: ' '.join([parte.strip() for parte in x.split(',')[::-1]]))

  # Novo dataframe só com jogadores brasileiros
  df_bra = df[df["Country"] == "BRA"]

  # Renomeando várias colunas
  df_bra = df_bra.rename(columns={
      'Name': 'nome',
      'Country': 'pais',
      'Num': 'numeroCamiseta',
      'B/T': 'batThrow',
      'Height': 'altura',
      'Weight': 'peso',
      'Organization': 'time',
      'Birth Date': 'dataNascimento'
  })


  # Exibe as primeiras linhas para verificação
  # print(df_bra.head())

  # Retorna concatenado
  return pd.concat([df_main, df_bra], ignore_index=True)

  # Fechar o driver
  driver.quit()

#### Tranforma os dados para data frame e exporta para Excel

In [26]:
# Exporta para um arquivo Excel
def export_to_excel(df, excel_file_name):
  df.to_excel(excel_file_name, index=False)

  print("Dados exportados para Excel com sucesso!")

#### Definição da URL para extrair

#### Extraindo todos os Roster

In [27]:
# Extraindo elenco (2022 e 2025)
df_main = extract_over_2022('https://www.mlb.com/world-baseball-classic/roster/brazil', 2025)
df_main = extract_over_2022('https://www.mlb.com/world-baseball-classic/roster/brazil?season=2022', 2022)
# df_main = extract_over_2022('https://www.mlb.com/world-baseball-classic/roster/colombia', 2025)

# Extraindo elenco (2016)
df_main = extract_equal_2016('https://web.archive.org/web/20161027043152/http://www.worldbaseballclassic.com/team?team=bra', 2016)

# Extraindo elenco (2012 e 2013)
df_main = extract_before_2013('https://web.archive.org/web/20130524210412/http://web.worldbaseballclassic.com/wbc/2013/rosters/index.jsp?season=2012', 2012)
df_main = extract_before_2013('https://web.archive.org/web/20130524210412/http://web.worldbaseballclassic.com/wbc/2013/rosters/index.jsp?season=2013', 2013)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

                 nome pais numeroCamiseta                 time batThrow  \
16    Gabriel Asakura  BRA             21                   --      R/R   
54       Felipe Burin  BRA              4  Los Angeles Dodgers      L/R   
75       Noris Chacon  BRA             28                   --      R/R   
127      Alan Fanhoni  BRA             33                   --      L/R   
128  Rafael Fernandes  BRA             27                   --      R/R   

    altura    peso dataNascimento  anoTorneio  
16    6'0"  185lbs     11/11/1988        2012  
54   5'10"  170lbs     02/10/1992        2012  
75    6'0"  209lbs     11/24/1972        2012  
127   6'4"  220lbs     12/23/1991        2012  
128  5'11"  170lbs     04/25/1986        2012  


<IPython.core.display.Javascript object>

                 nome pais numeroCamiseta                 time batThrow  \
18    Gabriel Asakura  BRA             21                   --      R/R   
47       Felipe Burin  BRA              4  Los Angeles Dodgers      L/R   
121      Alan Fanhoni  BRA             33                   --      L/R   
124  Rafael Fernandes  BRA             27                   --      R/R   
133      Diego Franca  BRA             12                   --      R/R   

    altura    peso dataNascimento  anoTorneio  
18    6'0"  185lbs     11/11/1988        2013  
47   5'10"  170lbs     02/10/1992        2013  
121   6'4"  220lbs     12/23/1991        2013  
124  5'11"  170lbs     04/25/1986        2013  
133   5'4"  159lbs     04/06/1987        2013  


#### Tratamento e Conversões

In [28]:
df_main = feet_to_m(df_main, 'altura', 'altura_m')  # Altura em metros
df_main = lbs_to_kg(df_main, 'peso', 'peso_kg')     # Peso em kg

df_main['bat'] = df_main['batThrow'].str[0]
df_main['throw'] = df_main['batThrow'].str[-1]

# Converte a coluna de data de nascimento para datetime
df_main['dataNascimento'] = pd.to_datetime(df_main['dataNascimento'])

# Normaliza a coluna de posições, removendo o "S" no final
df_main['posicao'] = df_main['posicao'].str.replace(r's$', '', regex=True)

# Calcula a idade no ano do campeonato
df_main['idade'] = df_main['anoTorneio'] - df_main['dataNascimento'].dt.year

# Identificando o tipo do torneio com base no ano
  # Q - Qualifier, M - Main
df_main.loc[df_main['anoTorneio'] != 2013, 'torneio'] = 'Q'
df_main.loc[df_main['anoTorneio'] == 2013, 'torneio'] = 'M'

In [29]:
df_main

,anoTorneio,pais,nome,posicao,time,numeroCamiseta,batThrow,altura,peso,dataNascimento,localNascimento,altura_m,peso_kg,bat,throw,idade,torneio
0,2025,NaN,Tiago Da Silva,Pitcher,NaN,22,R/R,"5' 9""",180,NaT,NaN,1.75,81.6,R,R,NaN,Q
1,2025,NaN,Murilo Gouvea,Pitcher,NaN,34,R/R,"6' 3""",210,NaT,NaN,1.91,95.3,R,R,NaN,Q
2,2025,NaN,Hugo Kanabushi,Pitcher,NaN,66,L/L,"5' 11""",180,NaT,NaN,1.80,81.6,L,L,NaN,Q
3,2025,NaN,Pedro Da Costa Lemos,Pitcher,NaN,38,R/R,"6' 0""",179,NaT,NaN,1.83,81.2,R,R,NaN,Q
4,2025,NaN,Joao Gabriel Marostica,Pitcher,NaN,21,R/R,"6' 3""",203,NaT,NaN,1.91,92.1,R,R,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,2013,BRA,Reinaldo Sato,NaN,--,36,R/R,"6'0""",195lbs,1980-10-25,NaN,1.83,88.5,R,R,33.0,M
151,2013,BRA,Marcio Tanaka,NaN,--,6,R/R,"5'8""",155lbs,1980-06-09,NaN,1.73,70.3,R,R,33.0,M
152,2013,BRA,Jean Tome,NaN,--,35,R/R,"6'2""",200lbs,1989-09-05,NaN,1.88,90.7,R,R,24.0,M
153,2013,BRA,Thyago Vieira,NaN,Seattle Mariners,32,R/R,"6'2""",210lbs,1993-01-07,NaN,1.88,95.3,R,R,20.0,M


#### Exportando resultado para um excel no drive

In [30]:
export_to_excel(df_main, '/content/drive/MyDrive/Baseball/rostersBrasil.xlsx')

Dados exportados para Excel com sucesso!
